In [21]:
import base64
import asyncio
from threading import Lock, Thread, Event
from queue import Queue, Empty

import cv2
import openai
from openai import AsyncOpenAI
from cv2 import VideoCapture, imencode
from dotenv import load_dotenv
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.messages import SystemMessage
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import AzureChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
from pyaudio import PyAudio, paInt16
from speech_recognition import Microphone, Recognizer, UnknownValueError

c:\Users\chaoqunzhao\.conda\envs\langchain\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
import os
import requests
import base64

# Configuration
GPT4V_KEY = "7d81f6e4806d4d5fbab47c339e0519d1"
IMAGE_PATH = r"C:\Users\chaoqunzhao\Downloads\testdata\testdata\persons\model\00017_00.jpg"
encoded_image = base64.b64encode(open(IMAGE_PATH, 'rb').read()).decode('ascii')
headers = {
    "Content-Type": "application/json",
    "api-key": GPT4V_KEY,
}

# Payload for the request
payload = {
  "messages": [
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": "You are an AI assistant that helps people find information."
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "Hi"
        }
      ]
    }
  ],
  "temperature": 0.7,
  "top_p": 0.95,
  "max_tokens": 800
}

GPT4V_ENDPOINT = "https://zcq1.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-02-15-preview"

# Send request
try:
    response = requests.post(GPT4V_ENDPOINT, headers=headers, json=payload)
    response.raise_for_status()  # Will raise an HTTPError if the HTTP request returned an unsuccessful status code
except requests.RequestException as e:
    raise SystemExit(f"Failed to make the request. Error: {e}")

# Handle the response as needed (e.g., print or process)
print(response.json())
response.json()['choices'][0]['message']['content']

{'choices': [{'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}, 'finish_reason': 'stop', 'index': 0, 'logprobs': None, 'message': {'content': 'Hello! How can I assist you today?', 'role': 'assistant'}}], 'created': 1722500564, 'id': 'chatcmpl-9rLN6lcEjAqBTm5yICN22CpGJ5Lo5', 'model': 'gpt-4o-2024-05-13', 'object': 'chat.completion', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'system_fingerprint': 'fp_abc28019ad', 'usage': {'completion_tokens': 9, 'prompt_tokens': 23, 'total_tokens': 32}}


'Hello! How can I assist you today?'

In [24]:
from langchain_openai import AzureChatOpenAI
model = AzureChatOpenAI(
    azure_deployment="gpt-4o",
    api_version="2024-05-01-preview",
    streaming=True)

messages = [
    (
        "system",
        "You are a helpful translator. Translate the user sentence to French.",
    ),
    ("human", "I love programming."),
]

stream = model.stream(messages)
full = next(stream)
for chunk in stream:
    full += chunk
    print(chunk)
full

content='J' id='run-dd826874-52ce-4d48-9667-d0bdd4852bc6'
content="'adore" id='run-dd826874-52ce-4d48-9667-d0bdd4852bc6'
content=' la' id='run-dd826874-52ce-4d48-9667-d0bdd4852bc6'
content=' programmation' id='run-dd826874-52ce-4d48-9667-d0bdd4852bc6'
content='.' id='run-dd826874-52ce-4d48-9667-d0bdd4852bc6'
content='' response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_abc28019ad'} id='run-dd826874-52ce-4d48-9667-d0bdd4852bc6'


AIMessageChunk(content="J'adore la programmation.", response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_abc28019ad'}, id='run-dd826874-52ce-4d48-9667-d0bdd4852bc6')

In [25]:
async def _stream_invoke(self, prompt, image):
        messages = [
            {"role": "system", "content": "You are a witty assistant that will use the chat history and the image provided by the user to answer its questions. Use few words on your answers. Go straight to the point. Do not use any emoticons or emojis. Do not ask the user any questions. Be friendly and helpful. Show some personality. Do not be too formal."},
            {"role": "user", "content": prompt},
            {"role": "user", "content": f"data:image/jpeg;base64,{image.decode()}"}
        ]
        
        # response = await model.astream(messages)

        async for chunk in model.astream(messages):
            if 'choices' in chunk:
                for choice in chunk['choices']:
                    if 'delta' in choice and 'content' in choice['delta']:
                        yield choice['delta']['content']

In [29]:
def _create_inference_chain(model):
        prompt_template = ChatPromptTemplate.from_messages(
            SystemMessage(content="Translate the following from English into Italian"),
            HumanMessage(content="hi!"),
        )

        chain = prompt_template | model | StrOutputParser()

        chat_message_history = ChatMessageHistory()
        return RunnableWithMessageHistory(
            chain,
            lambda _: chat_message_history,
            input_messages_key="prompt",
            history_messages_key="chat_history",
        )
llm = _create_inference_chain(model)
import uuid
session_id = str(uuid.uuid4())
ai_msg = llm.stream(messages, config={"configurable": {"session_id": session_id}})
for i in ai_msg:
    print(i)


AssertionError: The input to RunnablePassthrough.assign() must be a dict.